In [1]:
import torch
from torch.autograd import Variable
from model import UNet
import os
import numpy as np
import progressbar
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
from iterator import BasicVolumeIterator
from pred_utils import calc_pad_for_fit, calc_pad_for_pred_loss, crop_pad_width

In [3]:
patch_size = [116, 132, 132]
out_size = [28, 44, 44]

In [4]:
model = UNet(n_ch = 2, n_class = 1).cuda()
model.load_state_dict(torch.load('models/best_generator.pt'))

### Load the image and pad it so that we get same sized array after prediction

In [5]:
img_arr = np.load('/mnt/sdb1/intern_data/pix2pix_wbce_pet3d/test/0_input.npy')
pad_size1 = calc_pad_for_fit(img_arr.shape, out_size)
pad_size2 = calc_pad_for_pred_loss(patch_size, out_size)
pad_size_to_crop = pad_size1
tot_pad_size = [(x[0]+y[0], x[1]+y[1]) for x,y in zip(pad_size1, pad_size2)]
img_arr_padded = np.pad(img_arr, tot_pad_size, mode = 'constant')
dummy_output_arr_to_be_predicted = np.pad(img_arr, pad_size1, mode = 'constant')

In [6]:
inp_itr = BasicVolumeIterator(img_arr_padded, patch_size, out_size)
out_itr = BasicVolumeIterator(dummy_output_arr_to_be_predicted, out_size, out_size)

### Noise (to be sent to the generator)

In [7]:
def sample_noise(batch_size, channels = 1, width = 116, height = 132, depth = 132):
    z = torch.rand(batch_size, channels, width, height, depth)
    return z

### Raster scan the whole image and predict

In [8]:
bar = progressbar.ProgressBar(max_value = out_itr.get_num_patches())
count = 0
while inp_itr.is_not_over():
    input_arr = inp_itr.get_patch().reshape(1,1,116,132,132)
    inp_itr.move_coords()
    input_arr = Variable(torch.from_numpy(input_arr).float().cuda(), volatile = True)
    gen_noise = Variable(sample_noise(1).float().cuda(), volatile = True)
    input_arr = torch.cat((input_arr, gen_noise), 1)
    pred_arr = model(input_arr)
    pred_arr = pred_arr.data.cpu().numpy()
    pred_arr = pred_arr.reshape(28,44,44)
#    pred_arr = np.random.randn(28,44,44)
    out_itr.set_patch(pred_arr)
    out_itr.move_coords()
    bar.update(count)
    count = count + 1
output_arr = out_itr.vol_array
output_arr = crop_pad_width(output_arr, pad_size1)
np.save('experiments/0_amit_pred_best_generator.npy', output_arr)

 99% (1439 of 1440) |#################### | Elapsed Time: 0:05:53 ETA:  0:00:00